# Train a Linear Regression Model with Watson Machine Learning 

Notebook created by Zeming Zhao on June, 2021

In this notebook, you will learn how to use the Watson Machine Learning Accelerator (WML-A) API and accelerate the processing of Linear Regression model on GPU with Watson Machine Learning Accelerator.

Linear Regression is a simple machine learning model where the response y is modelled by a linear combination of the predictors in X.

In this notebook we have three versions of Linear Regression model: scikit-learn version, cuML version and snapML version.

All three versions will be submitted onto WMLA. And we can compare the performance benifit of cuML and snapML version.

This notebook covers the following sections:

1. [Generate dataset for Linear Regression](#dataset)<br>

1. [Setup Linear Regression using SK-Learning](#skl-model)<br>

1. [Training the SK-Learning model on CPU with Watson Machine Learning Accelerator](#skl-cpu)<br>

1. [Setup Linear Regression using cuML](#cuml-model)<br>

1. [Training the cuML model on GPU with Watson Machine Learning Accelerator](#cuml-gpu)<br>

1. [Setup Linear Regression using snapML](#snapml-model)<br>

1. [Training the SnapML model on CPU with Watson Machine Learning Accelerator](#snapml-cpu)<br>

1. [Setup Linear Regression using snapML GPU](#snapml-model-gpu)<br>

1. [Training the SnapML model on GPU with Watson Machine Learning Accelerator](#snapml-gpu)<br>

## Preparations
### Prepare directory and file for writing Linear Regression engine.

In [1]:
from pathlib import Path
dataset_dir = f'/project_data/data_asset/dataset' 
dataset_base_name = f'linearRegression-dataset.py'
Path(dataset_dir).mkdir(exist_ok=True)
print("create dataset directory done.")

create dataset directory done.


In [2]:
from pathlib import Path
model_dir = f'/project_data/data_asset/models' 
model_base_name = f'linearRegression-main.py'
Path(model_dir).mkdir(exist_ok=True)
print("create model directory done.")

create model directory done.


<a id = "dataset"></a>
## Step 1 : Generate dataset for Linear Regression

In [3]:
dataset_main='generate-'+dataset_base_name

In [4]:
%%writefile {dataset_dir}/{dataset_main}

# ```
#(C) Copyright IBM Corp.  2019,2020. All Rights Reserved.
# ```
#
# ```
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ```

# Dataset preprocessing and training code based on the solution posted here:
# https://www.kaggle.com/tsaustin/mercari-price-recommendation

import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os
import argparse
import numpy as np
import scipy

work_dir = "/gpfs/mydatafs/datasets"
source_file = work_dir + "/train.tsv"
the_max_features = 0

# Define output paths for the train and test datasets
x_train_filename = work_dir +"/dataset-lr/X_train.npz"
x_test_filename = work_dir +"/dataset-lr/X_test.npz"
y_train_filename = work_dir +"/dataset-lr/y_train.npy"
y_test_filename = work_dir +"/dataset-lr/y_test.npy"

# Read the training dataset
df = pd.read_csv(source_file,sep="\t")
print(df.head())

# Remove the train_id feature column
df.set_index('train_id',inplace=True)
print(df.head())

# Compute the percentages of missing values
df_na = (df.isnull().sum() / len(df)) * 100
df_na = df_na.drop(df_na[df_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :df_na})
missing_data.head(20)

# Compute the number of products with price 0
len(df[df['price'] == 0])

print(df['price'].unique())

# Remove the products with price 0
df = df[df['price']>0]


# Replace the missing values with a given word 'missing'
df['brand_name'] = df['brand_name'].fillna('missing')
df['category_name'] = df['category_name'].fillna('missing')
df['item_description'] = df['item_description'].fillna('missing')
print(df.head())

# Change the data type for category_name, brand_name and item_condition_id
df['category_name'] = df['category_name'].astype('category')
df['item_condition_id'] = df['item_condition_id'].astype('category')
print(df.dtypes)

# Words cleaning function for the columns with long text
def clean_text(col):
    # remove non alpha characters
    col = col.str.replace("[\W]", " ") #a-zA-Z1234567890
    # all lowercase
    col = col.apply(lambda x: x.lower())
    return col

# Clean the text in the name, category name and item description features
df['name']=clean_text(df['name'])
df['category_name']=clean_text(df['category_name'])
df['item_description']=clean_text(df['item_description'])
print(df.head())


# create feature matrix for name, category_name and item description features
from sklearn.feature_extraction.text import CountVectorizer

# The text will be replaced with a vector of word counts for each of the columns

if the_max_features==0:
    cv = CountVectorizer(min_df=0.0001, max_df=0.1, stop_words='english')
    X_name = cv.fit_transform(df['name'])
    print(X_name.shape)

    cv = CountVectorizer()
    X_category = cv.fit_transform(df['category_name'])
    print(X_category.shape)

    cv = CountVectorizer(min_df=0.0001, max_df=0.1, stop_words='english')
    X_item_description = cv.fit_transform(df['item_description'])
    print(X_item_description.shape)

else:
    cv = CountVectorizer(min_df=0.0001, max_df=0.1, max_features=the_max_features, stop_words='english')
    X_name = cv.fit_transform(df['name'])
    print(X_name.shape)

    cv = CountVectorizer(max_features=the_max_features)
    X_category = cv.fit_transform(df['category_name'])
    print(X_category.shape)

    cv = CountVectorizer(min_df=0.0001, max_df=0.1, max_features=the_max_features, stop_words='english')
    X_item_description = cv.fit_transform(df['item_description'])
    print(X_item_description.shape)


# the brand name is label binarizer
# generate the feature matrix for the brand name feature
from sklearn.preprocessing import LabelBinarizer
if the_max_features==0:
    lb = LabelBinarizer(sparse_output=True)
    X_brand = lb.fit_transform(df['brand_name'])
else:
    lb = LabelBinarizer(sparse_output=True)
    X_brand = lb.fit_transform(df['brand_name'])

y = df['price']
df = df.drop('price',axis=1)
print(df.head())

# one-hot encoding for the item condition id (5 integer values) and shipping (2 integer values)
from scipy.sparse import csr_matrix
df['shipping'] = df['shipping'].astype('category')
X_condition_shipping = csr_matrix(pd.get_dummies(df[['item_condition_id','shipping']], sparse=True).values)

# create the complete feature matrix by stacking the individual column transformations
from scipy.sparse import hstack

if the_max_features == 0:
    X_all = hstack((X_brand, X_category, X_name, X_item_description, X_condition_shipping)).tocsr()
else:
    X_all = hstack((X_brand, X_category, X_name, X_item_description, X_condition_shipping)).tocsr()
print(X_all.shape)

# reduce the feature columns by removing all features with a document frequency smaller than 1
import numpy as np
mask = np.array(np.clip(X_all.getnnz(axis=0) - 1, 0, 1), dtype=bool)
X_all = X_all[:, mask]
print(X_all.shape)

# target variable: the log of the price
y_log = np.log1p(y)

# perform standard scaling
print("Before Split\n")
print("X_all=",X_all.shape)
print("y_log=",y_log.shape)

# split into test and train samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_log, random_state=42, test_size=0.2)

print("After Split\n")
print("X_train=",X_train.shape)
print("X_test=",X_test.shape)
print("y_train=",y_train.shape)
print("y_test=",y_test.shape)

from sklearn.preprocessing import normalize
X_train = normalize(X_train, axis=1, norm='l1')
X_test = normalize(X_test, axis=1, norm='l1')

print("Saving X_train to ", x_train_filename)
print("Saving X_test to ", x_test_filename)
print("Saving y_train to ", y_train_filename)
print("Saving y_test to ", y_test_filename)

scipy.sparse.save_npz(x_train_filename, X_train, compressed=False)
scipy.sparse.save_npz(x_test_filename, X_test, compressed=False)
np.save(y_train_filename, y_train)
np.save(y_test_filename, y_test)


Overwriting /project_data/data_asset/dataset/generate-linearRegression-dataset.py


### Prepare the model lib for job submission

In [5]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from matplotlib import pyplot as plt
%pylab inline

import base64
import json
import time
import urllib

Populating the interactive namespace from numpy and matplotlib


### Configuring your environment and project details
To set up your project details, provide your credentials in this cell. You must include your cluster URL, username, and password.

In [6]:
# please enter Watson Machine Learning Accelerator host name
hostname='wmla-console-wmla.apps.dse-perf.cpolab.ibm.com'
# login='username:password' # please enter the login and password
login='mluser1:mluser1'
es = base64.b64encode(login.encode('utf-8')).decode("utf-8")
# print(es)
commonHeaders={'Authorization': 'Basic '+es}
req = requests.Session()
auth_url = 'https://{}/auth/v1/logon'.format(hostname)
print(auth_url)

a=requests.get(auth_url,headers=commonHeaders, verify=False)
access_token=a.json()['accessToken']
# print("Access_token: ", access_token)

dl_rest_url = 'https://{}/platform/rest/deeplearning/v1'.format(hostname)
commonHeaders={'accept': 'application/json', 'X-Auth-Token': access_token}
req = requests.Session()

# Health check
confUrl = 'https://{}/platform/rest/deeplearning/v1/conf'.format(hostname)
r = req.get(confUrl, headers=commonHeaders, verify=False)

https://wmla-console-wmla.apps.dse-perf.cpolab.ibm.com/auth/v1/logon


### Define the status checking function

In [7]:
import tarfile
import tempfile
import os
import json
import pprint
import pandas as pd
from IPython.display import clear_output

def query_job_status(job_id,refresh_rate=3) :

    execURL = dl_rest_url  +'/execs/'+ job_id['id']
    pp = pprint.PrettyPrinter(indent=2)

    keep_running=True
    res=None
    while(keep_running):
        res = req.get(execURL, headers=commonHeaders, verify=False)
        monitoring = pd.DataFrame(res.json(), index=[0])
        pd.set_option('max_colwidth', 120)
        clear_output()
        print("Refreshing every {} seconds".format(refresh_rate))
        display(monitoring)
        pp.pprint(res.json())
        if(res.json()['state'] not in ['PENDING_CRD_SCHEDULER', 'SUBMITTED','RUNNING']) :
            keep_running=False
        time.sleep(refresh_rate)
    return res

### Define the submission function

In [8]:
def submit_job_to_wmla (args, files) :
    starttime = datetime.datetime.now()
    r = requests.post(dl_rest_url+'/execs?args='+args, files=files,
                  headers=commonHeaders, verify=False)
    if not r.ok:
        print('submit job failed: code=%s, %s'%(r.status_code, r.content))
    job_status = query_job_status(r.json(),refresh_rate=5)
    endtime = datetime.datetime.now()
    print("\nTotallly training cost: ", (endtime - starttime).seconds, " seconds.")

<a id = "cpu"></a>
### Define the submission parameters for scikit-learn version on cpu

In [9]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType cpu \
--conda-env-name dlipy3-cpu  \
--model-main ' + dataset_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --conda-env-name dlipy3-cpu  --model-main generate-linearRegression-dataset.py


### Submit WMLA Workload

In [10]:
files = {'file': open("{0}/{1}".format(dataset_dir,dataset_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-1291,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --...,wmla-1291,mluser1,FINISHED,wmla-1291,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1291/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T10:10:26Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-1291',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu '
          '--conda-env-name dlipy3-cpu  --model-main '
          'generate-linearRegression-dataset.py ',
  'createTime': '2021-08-06T10:10:26Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-1291',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-1291',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1291/_submitted_code'}

Totallly training cost:  209  seconds.


<a id = "skl-model"></a>
## Step 2 : Setup Linear Regression using SK-Learning

In [11]:
model_main='sklean-'+model_base_name

In [12]:
%%writefile {model_dir}/{model_main}

import os, datetime
#from sklearn.datasets import make_regression
#from sklearn.model_selection import train_test_split
#from sklearn.datasets import make_sparse_uncorrelated
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression
import numpy as np
from scipy.sparse import load_npz

# load Data
data_dir =  "/gpfs/mydatafs/datasets/dataset-lr"
start = datetime.datetime.now()
X_train = load_npz(data_dir + "/X_train.npz")
X_test  = load_npz(data_dir + "/X_test.npz")
y_train = np.load(data_dir + "/y_train.npy")
y_test = np.load(data_dir + "/y_test.npy")
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

print(type(X_train))
print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

# scikit-learn Model
lr_model = skLinearRegression(fit_intercept=True,
                            normalize=True,
                            n_jobs=-1)

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

# Predict
start = datetime.datetime.now()
predict = lr_model.predict(X_test)
end = datetime.datetime.now()
print ("predict timecost: %.2gs" % ((end-start).total_seconds()))

# Evaluate
start = datetime.datetime.now()
r2_score = r2_score(y_test, predict)
end = datetime.datetime.now()
print ("evaluate timecost: %.2gs" % ((end-start).total_seconds()))

#  print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (scikit-learn): %.4f" % r2_score)

Overwriting /project_data/data_asset/models/sklean-linearRegression-main.py


<a id = "skl-cpu"></a>
## Step 3 :  Training the SK-Learning model on CPU with Watson Machine Learning Accelerator


In [13]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 16G \
--workerDeviceType cpu \
--conda-env-name dlipy3-cpu  \
--model-main ' + model_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu --conda-env-name dlipy3-cpu  --model-main sklean-linearRegression-main.py


In [14]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-1292,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu --...,wmla-1292,mluser1,FINISHED,wmla-1292,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1292/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T10:13:56Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-1292',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu '
          '--conda-env-name dlipy3-cpu  --model-main '
          'sklean-linearRegression-main.py ',
  'createTime': '2021-08-06T10:13:56Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-1292',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-1292',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1292/_submitted_code'}

Totallly training cost:  161  seconds.


<a id = "cuml-model"></a>
## Step 4 :  Setup Linear Regression using cuML

In [15]:
model_main='cuML-'+model_base_name

In [16]:
%%writefile {model_dir}/{model_main}
####
####  please reduce the dataset size if your GPU cannot handle the input dataset
####

import os, datetime
import cudf
#from cuml import make_regression, train_test_split
#from sklearn.datasets import make_sparse_uncorrelated
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
import numpy as np
from scipy.sparse import load_npz
import pandas as pd
import gc

cudf.set_allocator("managed")

# specify the cache location to /gpfy since ~/.cache is not available
os.environ["CUPY_CACHE_DIR"]="/gpfs/mydatafs/models/cache/lr"

# load Data
data_dir =  "/gpfs/mydatafs/datasets/dataset-lr"
start = datetime.datetime.now()
X_train_ = load_npz(data_dir + "/X_train.npz")
print(X_train_.shape)
#X_train__ = pd.DataFrame.sparse.from_spmatrix(X_train_)
X_train = pd.DataFrame.sparse.from_spmatrix(X_train_)
del X_train_
gc.collect()
#X_train = cudf.DataFrame(X_train__[-200000:])
#del X_train__
#gc.collect()
X_test_ = load_npz(data_dir + "/X_test.npz")
print(X_test_.shape)
#X_test__ = pd.DataFrame.sparse.from_spmatrix(X_test_)
X_test = pd.DataFrame.sparse.from_spmatrix(X_test_)
del X_test_
gc.collect()
#X_test = cudf.DataFrame(X_test__[-50000:])
#del X_test__
#gc.collect()
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

start = datetime.datetime.now()
# X_train = np.load(data_dir + "/X_train.npy")
# X_test  = np.load(data_dir + "/X_test.npy")
y_train = np.load(data_dir + "/y_train.npy")
#y_train = y_train[-200000:]
y_test = np.load(data_dir + "/y_test.npy")
#y_test = y_test[-50000:]
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

# cuML Model
lr_model = cuLinearRegression(fit_intercept=True,normalize=True,algorithm='eig')

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

# Predict
start = datetime.datetime.now()
predict_cuml = lr_model.predict(X_test)
end = datetime.datetime.now()
print ("predict timecost: %.2gs" % ((end-start).total_seconds()))

# Evaluate
start = datetime.datetime.now()
r2_score_cuml = r2_score(y_test, predict_cuml)
end = datetime.datetime.now()
print ("evaluate timecost: %.2gs" % ((end-start).total_seconds()))

print("R^2 score (cuML): %.4f" % r2_score_cuml)

Overwriting /project_data/data_asset/models/cuML-linearRegression-main.py


<a id = "cuml-gpu"></a>
## Step 5 :  Training the cuML model on GPU with Watson Machine Learning Accelerator
### Re-define the submssion parameters

In [17]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 16G \
--workerDeviceType gpu \
--conda-env-name rapids-21.06-new \
--model-main ' + model_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu --conda-env-name rapids-21.06-new --model-main cuML-linearRegression-main.py


### Submit WMLA Workload

In [18]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,events,nameSpace,numWorker,framework
0,wmla-1293,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu --...,wmla-1293,mluser1,ERROR,wmla-1293,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1293/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T10:16:37Z,False,Job <wmla-1293> task <1> failed,wmla,1,tensorflow


{ 'appId': 'wmla-1293',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu '
          '--conda-env-name rapids-21.06-new --model-main '
          'cuML-linearRegression-main.py ',
  'createTime': '2021-08-06T10:16:37Z',
  'creator': 'mluser1',
  'elastic': False,
  'events': 'Job <wmla-1293> task <1> failed',
  'framework': 'tensorflow',
  'id': 'wmla-1293',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'ERROR',
  'submissionId': 'wmla-1293',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1293/_submitted_code'}

Totallly training cost:  62  seconds.


<a id = "snapml-model"></a>
## Step 6:  Setup Linear Regression model using snapML

In [19]:
model_main='snapML-'+model_base_name

In [20]:
%%writefile {model_dir}/{model_main}

import datetime
#from sklearn.datasets import make_regression
#from sklearn.model_selection import train_test_split
#from sklearn.datasets import make_sparse_uncorrelated
from sklearn.metrics import r2_score
from snapml import LinearRegression as SnapLinearRegression
import numpy as np
from scipy.sparse import load_npz

# load Data
data_dir =  "/gpfs/mydatafs/datasets/dataset-lr"
start = datetime.datetime.now()
X_train = load_npz(data_dir + "/X_train.npz")
X_test  = load_npz(data_dir + "/X_test.npz")
y_train = np.load(data_dir + "/y_train.npy")
y_test = np.load(data_dir + "/y_test.npy")
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

# snapML model
lr_model = SnapLinearRegression(fit_intercept=True,normalize=True)

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

r2_score_snapml = r2_score(y_test, lr_model.predict(X_test))
print("R^2 score (snapML): %.4f" % r2_score_snapml)

Overwriting /project_data/data_asset/models/snapML-linearRegression-main.py


<a id = "snapml-cpu"></a>
## Step 7 :  Training the SnapML model on CPU with Watson Machine Learning Accelerator
### Re-define the submission parameters

In [21]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 16G \
--workerDeviceType cpu \
--conda-env-name snapml-177rc \
--model-main ' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu --conda-env-name snapml-177rc --model-main snapML-linearRegression-main.py


### Submit WMLA Workload

In [22]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-1294,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu --...,wmla-1294,mluser1,FINISHED,wmla-1294,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1294/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T10:17:40Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-1294',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 16G --workerDeviceType cpu '
          '--conda-env-name snapml-177rc --model-main '
          'snapML-linearRegression-main.py ',
  'createTime': '2021-08-06T10:17:40Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-1294',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-1294',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1294/_submitted_code'}

Totallly training cost:  34  seconds.


<a id = "snapml-model-gpu"></a>
## Step 8 :  Setup Linear Regression model using snapML on GPU
### Create a Linear Regression Model based on snapML 

In [23]:
model_main='snapML-GPU-'+model_base_name

In [24]:
%%writefile {model_dir}/{model_main}

import datetime
#from sklearn.datasets import make_regression
#from sklearn.model_selection import train_test_split
#from sklearn.datasets import make_sparse_uncorrelated
from sklearn.metrics import r2_score
from snapml import LinearRegression as SnapLinearRegression
import numpy as np
from scipy.sparse import load_npz

# load Data
data_dir =  "/gpfs/mydatafs/datasets/dataset-lr"
start = datetime.datetime.now()
X_train = load_npz(data_dir + "/X_train.npz")
X_test  = load_npz(data_dir + "/X_test.npz")
y_train = np.load(data_dir + "/y_train.npy")
y_test = np.load(data_dir + "/y_test.npy")
end = datetime.datetime.now()
print ("load data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

# snapML model
lr_model = SnapLinearRegression(use_gpu=True,
                                fit_intercept=True,normalize=True)

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

r2_score_snapml = r2_score(y_test, lr_model.predict(X_test))
print("R^2 score (snapML): %.4f" % r2_score_snapml)

Overwriting /project_data/data_asset/models/snapML-GPU-linearRegression-main.py


<a id = "snapml-gpu"></a>
## Step 9 :  Training the SnapML model on GPU with Watson Machine Learning Accelerator
### Re-define the submission parameters

In [25]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 16G \
--workerDeviceType gpu \
--conda-env-name snapml-177rc \
--model-main ' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu --conda-env-name snapml-177rc --model-main snapML-GPU-linearRegression-main.py


In [26]:
files = {'file': open("{0}/{1}".format(model_dir,model_main),'rb')}
submit_job_to_wmla (args, files)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-1295,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu --...,wmla-1295,mluser1,FINISHED,wmla-1295,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-1295/_submitted_code,SingleNodeTensorflowTrain,2021-08-06T10:18:14Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-1295',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 16G --workerDeviceType gpu '
          '--conda-env-name snapml-177rc --model-main '
          'snapML-GPU-linearRegression-main.py ',
  'createTime': '2021-08-06T10:18:14Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-1295',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-1295',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-1295/_submitted_code'}

Totallly training cost:  46  seconds.
